# Midterm Project Part 2
# Predicting Bike Sharing Demand

We will test the k-Nearest-Neighbor (kNN) method for predicting the bike sharing demand. The core idea of the kNN method is to make prediction on a given day based on k closest records from the dataset. The main challenge of using this method is to find a good way to determine "closeness".

From the analysis in Part 1, we know that the number of bike rentals correlates with year, season, weekday, working day, weather, temperature, humidity, and windspeed. However, any single feature is not powerful enough to provide a good prediction. We need to create a similarity measure that combines the closeness in all relevant features.

## I. Create a Test Set

We will randomly extract 3 records from the dataset as the test set to evaluate the performance of the kNN method.

1. Load the `day.csv` as a data frame

2. Use `numpy.random.choice()` to select 3 row indices. Print the 3 selected indices. Read the [documentation page](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) for the details of this function.

3. Create a test data frame using the selected rows. Display the test data frame.

4. Update the `data` data frame by dropping the 3 test rows. Display the shape of the updated data frame.

## II. Create a Similarity Measure

A reasonable similarity measure between two data records should achieve the following:
- It combines the differences on each relevant feature: season, yr, mnth, holiday, weekday, workingday, weathersit, temp, hum, windspeed.
- Use absolute value of the differences to prevent cancelation between positive differences and negative differences.
- Use the reciprocal of feature standard deviations as the weights in the aggregation.

1. Calculate the standard deviation of each relavent feature. Store the reciprocals of the standard deviations in a `weights` list and print the list.

2. Create a `similarity(test_row, data_row, weights)` function that does the following:
- Calculate the absolute value of the difference between `test_row` and `data_row` on each relevant feature.
- Calculate the sum of all differences, each multiplied with its corresponding weight.
- Return the sum

3. Test the `similarity` function by calculating the similarity between the first row in the test data frame and the first row in `data`.

## III Test the kNN Method

Apply the kNN method with k=5 on each of the test rows.

1. In `data`, add a new column named `'sim1'` that shows the similarity between each data row with the first test row. Display the first 5 rows of `data`.

2. Sort the rows according to their `sim1` value, and extract 5 rows with the smallest `sim1` values. Display these rows.

3. Display the average number of renters from these 5 rows. The number of bike renters is the value in the 'cnt' column. This is kNN's prediction on the first test row.

4. Display the number of renters from the first test row.

5. Repeat Step 1-4 for the other two test rows. Display kNN's predictions and the actual number of renters.